# Project on Predicting Four Different Stages of Alzheimer's Disease
  * Non Demented
  * Very Mild Demented
  * Mild Demented
  * Moderate Demented

## Importing Required Libraries

In [2]:
## General Libraries

import numpy as np
import matplotlib.pyplot as plt

## Image Visualization and Preprocessing

import os
import PIL
import cv2
import pathlib

## Deep Learning Libraries and Modules

import tensorflow as tf
import tf_keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

## Model Evaluation Modules

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


ModuleNotFoundError: No module named 'tf_keras'

## Converting Images into Data from Directory

In [ ]:
## !unzip "/content/drive/MyDrive/Alzheimer's Disease/archive (13).zip" -d "/content/drive/MyDrive/Alzheimer's Disease"

In [ ]:
datadir = "AugmentedAlzheimerDataset/"

In [ ]:
datadir = pathlib.Path(datadir)
datadir

In [ ]:
len(list(datadir.glob('*/*.jpg')))

In [ ]:
len(list(datadir.glob('*/*.jpeg')))

In [ ]:
len(list(datadir.glob('*/*.png')))

In [ ]:
list(datadir.glob('*/*.jpg'))[:10]

In [ ]:
image_count = len(list(datadir.glob('*/*.jpg')))
image_count

## Viewing some Images using PIL  

In [ ]:
veryMildDemented = list(datadir.glob('VeryMildDemented/*'))
veryMildDemented[:5]

In [ ]:
PIL.Image.open(veryMildDemented[0])

In [ ]:
PIL.Image.open(veryMildDemented[1])

In [ ]:
MildDemented = list(datadir.glob('MildDemented/*'))
MildDemented[:5]

In [ ]:
PIL.Image.open(MildDemented[0])

In [ ]:
NonDemented = list(datadir.glob('NonDemented/*'))
NonDemented[:5]

In [ ]:
PIL.Image.open(NonDemented[0])

In [ ]:
ModerateDemented = list(datadir.glob('ModerateDemented/*'))
ModerateDemented[:5]

In [ ]:
PIL.Image.open(ModerateDemented[0])

## Converting Images into Numbers and Labelling

In [ ]:
images_dict = {
    'NonDemented' : list(datadir.glob('NonDemented/*')),
    'VeryMildDemented' : list(datadir.glob('VeryMildDemented/*')),
    'MildDemented' : list(datadir.glob('MildDemented/*')),
    'ModerateDemented' : list(datadir.glob('ModerateDemented/*')),
}

In [ ]:
labels_dict = {
    'NonDemented' : 0,
    'VeryMildDemented' : 1,
    'MildDemented' : 2,
    'ModerateDemented' : 3,
}

In [ ]:
images_dict['MildDemented'][:5]

### Using opencv library to convert data into numbers and labelling them

In [ ]:
# x, y = [], []

# for disease, images in images_dict.items():
#   for image in images:
#     img = cv2.imread(str(image))
#     resized_img = cv2.resize(img,(180,180))
#     x.append(resized_img)
#     y.append(labels_dict[disease])

In [ ]:
len(images_dict['MildDemented'])

In [ ]:
len(images_dict['VeryMildDemented'])

In [ ]:
len(images_dict['ModerateDemented'])

In [ ]:
len(images_dict['NonDemented'])

In [ ]:
8960 + 8960 + 6464 + 9600

In [ ]:
def process_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [180, 180])
    return image, label

def create_dataset(image_paths, labels, batch_size=32, shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.shuffle(buffer_size=shuffle_buffer_size)
    dataset = dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

image_paths = [str(image) for disease, images in images_dict.items() for image in images]
labels = [labels_dict[disease] for disease, images in images_dict.items() for _ in images]

train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, stratify=labels, random_state=42)

train_dataset = create_dataset(train_paths, train_labels, batch_size=32, shuffle_buffer_size=len(train_paths))
val_dataset = create_dataset(val_paths, val_labels, batch_size=32, shuffle_buffer_size=len(val_paths))

In [ ]:
model = Sequential([
    Input(shape=(180,180,3)),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(256, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(512, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
early_stopping_cb = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(train_dataset, validation_data=val_dataset, epochs=10,
          callbacks=[checkpoint_cb, early_stopping_cb])